## Preprocessing Tutorial (EOG Regression)
Derek Rosenzweig

### Loading Raw EEG Data

In [1]:
import mne
import numpy as np
import pandas as pd
import os
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Set parameters for fif path
sub = 'pilot-2'
stim = 'Jobs1'
seg = 'segment_1'

base_path = '/Users/derekrosenzweig/Documents/GitHub/EEG-Speech'
word_path = f'{base_path}/annotations/words/tsv/{stim}-words.tsv'
phoneme_path = f'{base_path}/annotations/phonemes/tsv/{stim}-phonemes.tsv'
fif_path = f'{base_path}/segmented_data/pilot-2/{sub}_{seg}_{stim}.fif'

# Load the data in MNE
raw = mne.io.read_raw_fif(fif_path, preload=True)

# Get sampling rate
sampling_rate = raw.info['sfreq']

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Opening raw data file /Users/derekrosenzweig/Documents/GitHub/EEG-Speech/segmented_data/pilot-2/pilot-2_segment_1_Jobs1.fif...
    Range : 0 ... 312072 =      0.000 ...   312.072 secs
Ready.


/var/folders/k2/2786kgx974vf04xkmg2tpfhr0000gr/T/ipykernel_43688/2290598829.py:20: RuntimeWarning: This filename (/Users/derekrosenzweig/Documents/GitHub/EEG-Speech/segmented_data/pilot-2/pilot-2_segment_1_Jobs1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload=True)


Reading 0 ... 312072  =      0.000 ...   312.072 secs...


### Apply Filters and Common Average Reference
#### This section applies various filters and re-referencing techniques to the raw EEG data.

- Applying a notch filter to remove power line interference (60 Hz) 
- Applying a bandpass filter (.1 - 40 Hz) 
- Re-referencing the data using reference channel 'VREF' 
- Selecting only the EEG channels
- Setting specific channels as electrooculography (EOG) channels
- Applying the common average reference 
- Extracting EEG data after common average re-referencing

In [2]:
# Set Notch filter
raw.notch_filter(60)

# Set filter
raw.filter(l_freq=.1, h_freq=40.0)

# Re-reference the data using the 'VREF' channel
raw.set_eeg_reference(['VREF'])

# Extract only the channels starting with 'E'
eeg_channels = [ch for ch in raw.ch_names if ch.startswith('E')]
raw = raw.pick_channels(eeg_channels)

# Change channel types to 'eog'
eog_channels = ['E126', 'E127']  
# eog_channels = ['E126', 'E127', 'E17', 'E21', 'E14', 'E25', 'E22', 'E15', 'E9', 'E8', 'E10', 'E18'] 

raw.set_channel_types({ch: 'eog' for ch in eog_channels})

# Apply common average reference
raw_car = raw.set_eeg_reference('average', projection=True)

# Get data from common average reference 
car_eeg_data = raw_car._data

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 6601 samples (6.601 s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


### Load Events and Create Epochs
#### This section loads word annotation metadata and creates epochs based on word onsets in the EEG data.

The main steps include:
- Loading word annotation metadata from a TSV file
- Creating events based on the word onsets in the EEG data
- Defining epoch parameters (time window and baseline correction interval)
- Creating epochs from the raw EEG data using the word onset events
- Accessing and printing the shape of the epoched EEG data

In [3]:
# Load metadata for words from annotations
word_info = pd.read_csv(word_path, delimiter='\t', encoding='utf-8')

# Create events for word epochs
word_onsets = (word_info['Start'].values * sampling_rate).astype(int)
event_id = {'word': 1}
events = np.column_stack((word_onsets, np.zeros_like(word_onsets), np.ones_like(word_onsets)))

# Define the epoch parameters
tmin = -0.2 
tmax = 0.6  
# baseline = (-0.2, 0)  # Baseline correction interval

# Create epochs based on the word onsets
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, preload=True)

# Access the epoched data
epoched_data = epochs.get_data(copy=True)
print(f"Epoched data shape: {epoched_data.shape}")


Not setting metadata
825 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 825 events and 801 original time points ...
0 bad epochs dropped
Epoched data shape: (825, 128, 801)


In [4]:
# Create word epochs
word_onsets = (word_info['Start'].values * sampling_rate).astype(int)
word_events = np.column_stack((word_onsets, np.zeros_like(word_onsets), np.ones_like(word_onsets)))
word_epochs = mne.Epochs(raw_car, word_events, tmin=-0.2, tmax=.6, baseline=None, reject=None, flat=None)
word_epochs.metadata = word_info  # Assign metadata to word epochs

# Define the directory path for saving word epochs
word_epochs_dir = os.path.join(base_path, 'derivatives', 'individual','word_epochs')
os.makedirs(word_epochs_dir, exist_ok=True)

# Save word epochs with a specific filename
word_epochs_filename = f'word-epo-{sub}-{stim}-{seg}-epo.fif'
word_epochs_filepath = os.path.join(word_epochs_dir, word_epochs_filename)
word_epochs.save(word_epochs_filepath, overwrite=True)
print(f"Word epochs saved to: {word_epochs_filepath}")

# Average epochs for evoked response
word_evoked = word_epochs.average()

Not setting metadata
825 matching events found
No baseline correction applied
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Adding metadata with 93 columns
Overwriting existing file.
Using data from preloaded Raw for 825 events and 801 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 801 original time points ...
Overwriting existing file.
Using data from preloaded Raw for 825 events and 801 original time points ...
Word epochs saved to: /Users/derekrosenzweig/Documents/GitHub/EEG-Speech/derivatives/individual/word_epochs/word-epo-pilot-2-Jobs1-segment_1-epo.fif


## Visualize Evoked Response

In [5]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Plot the evoked response
word_evoked.plot_joint()

# Show the plot
plt.show()


Projections have already been applied. Setting proj attribute to True.


<IPython.core.display.Javascript object>

### Plot Regression Coefficients for EOG Channels 

In [6]:
# Create word epochs
word_onsets = (word_info['Start'].values * sampling_rate).astype(int)
word_events = np.column_stack((word_onsets, np.zeros_like(word_onsets), np.ones_like(word_onsets)))
word_epochs = mne.Epochs(raw_car, word_events, tmin=-0.2, tmax=0.6, baseline=None, reject=None, flat=None, preload=True)
word_epochs.metadata = word_info  # Assign metadata to word epochs

# Run EOG regression
from mne.preprocessing import EOGRegression
# https://mne.tools/dev/generated/mne.preprocessing.EOGRegression.html
# https://www.sciencedirect.com/science/article/pii/S0987705300000551?via%3Dihub


# Perform regression using the EOG sensor as independent variable and the EEG sensors as dependent variables.
model_plain = EOGRegression(picks="eeg", picks_artifact="eog").fit(word_epochs)

# Plot regression coefficients (EOG channels) as topomap
fig_regression = model_plain.plot(vlim=(None, 0.4))
fig_regression.set_size_inches(3, 2)
plt.show()

Not setting metadata
825 matching events found
No baseline correction applied
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 825 events and 801 original time points ...
0 bad epochs dropped
Adding metadata with 93 columns
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<IPython.core.display.Javascript object>

### Subtract EOG Signal From EEG 

In [7]:
%matplotlib notebook
import matplotlib.pyplot as plt

epochs_clean_plain = model_plain.apply(word_epochs)
# After regression, redo the baseline correction
epochs_clean_plain.apply_baseline()
# Show evoked potential computed on the corrected data
evoked_clean_plain = epochs_clean_plain.average()

# Plot the evoked response
evoked_clean_plain.plot_joint()

# Show the plot
plt.show()

Applying baseline correction (mode: mean)
Projections have already been applied. Setting proj attribute to True.


<IPython.core.display.Javascript object>

### Load Phoneme Events and Create Epochs
#### This section loads phoneme annotation metadata and creates epochs based on phoneme onsets in the EEG data.


- Loading phoneme annotation metadata from a TSV file
- Creating events based on the phoneme onsets in the EEG data
- Defining epoch parameters (time window and baseline correction interval)
- Creating epochs from the raw EEG data using the phoneme onset events
- Accessing and printing the shape of the phoneme epoched EEG data


In [31]:
# Load metadata for words from annotations
phoneme_info = pd.read_csv(phoneme_path, delimiter='\t', encoding='utf-8')

# Create events for word epochs
phoneme_onsets = (phoneme_info['Start'].values * sampling_rate).astype(int)
event_id = {'phoneme': 1}
events = np.column_stack((phoneme_onsets, np.zeros_like(phoneme_onsets), np.ones_like(phoneme_onsets)))

# Define the epoch parameters
tmin = -0.1 
tmax = 0.3  
# baseline = (-0.2, 0)  # Baseline correction interval

# Create epochs based on the word onsets
phoneme_epochs = mne.Epochs(raw, events, event_id, tmin, tmax, preload=True)


# Access the epoched data
phoneme_epoched_data = phoneme_epochs.get_data(copy=True)
print(f"Phoneme epoched data shape: {phoneme_epoched_data.shape}")

Not setting metadata
2932 matching events found
Setting baseline interval to [-0.1, 0.0] s
Applying baseline correction (mode: mean)
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 2932 events and 401 original time points ...
0 bad epochs dropped
Phoneme epoched data shape: (2932, 128, 401)


### Add Phoneme Metadata and Average Response to Phoneme Onset 

In [32]:
phoneme_onsets = (phoneme_info['Start'].values * sampling_rate).astype(int)
phoneme_events = np.column_stack((phoneme_onsets, np.zeros_like(phoneme_onsets), np.ones_like(phoneme_onsets)))
phoneme_epochs = mne.Epochs(raw_car, phoneme_events, tmin=-0.1, tmax=.3, baseline=None, reject=None, flat=None)
phoneme_epochs.metadata =  phoneme_info  # Assign metadata to phoneme epochs

# Define the directory path for saving phoneme epochs
phoneme_epochs_dir = os.path.join(base_path, 'derivatives', 'individual','phoneme_epochs')
os.makedirs(phoneme_epochs_dir, exist_ok=True)

# Save phoneme epochs with a specific filename
phoneme_epochs_filename = f'phoneme-epo-{sub}-{stim}-{seg}-epo.fif'
phoneme_epochs_filepath = os.path.join(phoneme_epochs_dir, phoneme_epochs_filename)
phoneme_epochs.save(phoneme_epochs_filepath, overwrite=True)
print(f"Phoneme epochs saved to: {phoneme_epochs_filepath}")

# Average epochs for evoked response
phoneme_evoked = phoneme_epochs.average()

Not setting metadata
2932 matching events found
No baseline correction applied
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Adding metadata with 12 columns
Overwriting existing file.
Using data from preloaded Raw for 2932 events and 401 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 401 original time points ...
Overwriting existing file.
Using data from preloaded Raw for 2932 events and 401 original time points ...
Phoneme epochs saved to: /Users/derekrosenzweig/Documents/GitHub/EEG-Speech/derivatives/individual/phoneme_epochs/phoneme-epo-pilot-2-Jobs1-segment_1-epo.fif


In [19]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Plot the evoked response
phoneme_evoked.plot_joint()

# Show the plot
plt.show()

# Phoneme epochs epoch window size -3 +3 decoding 

Projections have already been applied. Setting proj attribute to True.


<IPython.core.display.Javascript object>

### Phoneme Decoding Analysis
#### This section performs a decoding analysis to classify different phoneme features.


In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold
import os
import csv


phoneme_epochs.load_data()  # Load data into memory
phoneme_epochs.resample(100, npad='auto')  # Downsample with automatic padding


# Sklearn wrapper for Logistic Regression
clf = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear'))
cv = KFold(5, shuffle=True)

# Specify the desired values for decoding
desired_phonation_value = 'v'
desired_manner_value = 'f'
desired_place_value = 'm'

# Filter epochs based on desired values for decoding
filtered_epochs_phonation = phoneme_epochs[phoneme_epochs.metadata['phonation'] == desired_phonation_value]
filtered_epochs_manner = phoneme_epochs[phoneme_epochs.metadata['manner'] == desired_manner_value]
filtered_epochs_place = phoneme_epochs[phoneme_epochs.metadata['place'] == desired_place_value]

# Concatenate filtered epochs from all features of interest
filtered_epochs = mne.concatenate_epochs([filtered_epochs_phonation, filtered_epochs_manner, filtered_epochs_place])

# Plotting
fig, ax = plt.subplots(1, figsize=(10, 5))
plt.title(f"Decoding Accuracy for {sub}")

for feat, label, color in zip(['phonation', 'manner', 'place'], ['Voiced', 'Fricatives', 'Pure Vowel'], ['purple', 'darkviolet', 'mediumorchid']):
    desired_value = {'phonation': desired_phonation_value, 'manner': desired_manner_value, 'place': desired_place_value}[feat]
    y = (filtered_epochs.metadata[feat] == desired_value).astype(int)

    # Preallocate accuracy scores array
    accuracy_scores = np.empty(filtered_epochs.get_data(copy=True).shape[-1])

    # Extract data and compute ROC-AUC scores across time
    for tt in range(accuracy_scores.shape[0]):
        X_ = filtered_epochs.get_data(copy=True)[:, :, tt]
        scores = cross_val_score(clf, X_, y, scoring='roc_auc', cv=cv, n_jobs=-1)
        accuracy_scores[tt] = scores.mean()

    ax.plot(filtered_epochs.times, accuracy_scores, label=label, color=color)

# Add vertical dashed grey line at t=0
ax.axvline(x=0, color='grey', linestyle='--')

# Add horizontal dashed line at y=0.5 (chance level decoding)
ax.axhline(y=0.5, color='grey', linestyle='--')

ax.set_xlabel("Time (ms) relative to phoneme onset")
ax.set_ylabel("ROC-AUC")
ax.legend()
plt.show()

Using data from preloaded Raw for 2932 events and 401 original time points ...
Adding metadata with 12 columns
2952 matching events found
No baseline correction applied
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)


<IPython.core.display.Javascript object>

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### EOG Regression on Phoneme Epochs 

Note -- shorter epochs seems to reduce impact of blink artifacts in signal 

In [16]:
# Create phoneme epochs
phoneme_onsets = (phoneme_info['Start'].values * sampling_rate).astype(int)
phoneme_events = np.column_stack((phoneme_onsets, np.zeros_like(phoneme_onsets), np.ones_like(phoneme_onsets)))
phoneme_epochs = mne.Epochs(raw_car, phoneme_events, tmin=-0.1, tmax=0.3, baseline=None, reject=None, flat=None, preload=True)
phoneme_epochs.metadata = phoneme_info  # Assign metadata to phoneme epochs

# Run EOG regression
from mne.preprocessing import EOGRegression
# https://mne.tools/dev/generated/mne.preprocessing.EOGRegression.html
# https://www.sciencedirect.com/science/article/pii/S0987705300000551?via%3Dihub


# Perform regression using the EOG sensor as independent variable and the EEG sensors as dependent variables.
model_plain = EOGRegression(picks="eeg", picks_artifact="eog").fit(phoneme_epochs)

# Plot regression coefficients (EOG channels) as topomap
fig_regression = model_plain.plot(vlim=(None, 0.4))
fig_regression.set_size_inches(3, 2)
plt.show()

Not setting metadata
2932 matching events found
No baseline correction applied
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 2932 events and 401 original time points ...
0 bad epochs dropped
Adding metadata with 12 columns
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<IPython.core.display.Javascript object>

In [33]:
%matplotlib notebook
import matplotlib.pyplot as plt

epochs_clean_plain = model_plain.apply(phoneme_epochs)
# After regression, redo the baseline correction
epochs_clean_plain.apply_baseline()
# Show evoked potential computed on the corrected data
evoked_clean_plain = epochs_clean_plain.average()

# Plot the evoked response
evoked_clean_plain.plot_joint()

# Show the plot
plt.show()

Applying baseline correction (mode: mean)
Projections have already been applied. Setting proj attribute to True.


<IPython.core.display.Javascript object>